# Twitter Analysis using Pyspark


In [ ]:
##import neccessary libraries and modules
from __future__ import print_function
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext, DataFrame
from pyspark.sql.functions import *

In [ ]:
##Create a spark instance
sc= pyspark.SparkContext("yarn")
sqlContext = SQLContext(sc)

In [ ]:
##read the json file with Twitter data
df = sqlContext.read.json('/data/ProjectDataset/statuses.log.2014-12-1*.gz')
##select the columns required for analysis
df_sub = df.na.drop(subset=["user.id"]).select(["user","entities", "lang", "retweeted", "favorited", "text","delete","retweeted_status"])

In [ ]:
##subset the dataframe with entities only
from pyspark.sql.functions import array_contains
entities_df = df_sub.select('entities.*')

##aggreate the number tweets for each hashtag
hashtags = entities_df.groupby('hashtags.text').count()
print(hashtags.sort(desc('count')).show())

In [1]:
##count the number of retweets
MTVStars_df = df_sub.filter(array_contains(df_sub.entities.hashtags.text,'MTVStars'))
MTVStars_retweeted_status = MTVStars_df.select('retweeted_status.*')
retweet_count = MTVStars_retweeted_status.groupby('retweet_count').count()

##sort in the descending order
print(retweet_count.sort(desc('count')).show())
retweet = MTVStars_retweeted_status.groupby('retweeted').count()
print(retweet.sort(desc('count')).show())

NameError: name 'df_sub' is not defined

In [ ]:
##subset the main dataframe for users only to see the users tweeting the most
MTVstars_user = MTVStars_df.select('user.*')
MTVstars_urls = MTVstars_user.groupby('url').count()


In [ ]:
##sort in the descending
print(MTVstars_urls.sort(desc('count')).show())
MTVstars_tweeters_name = MTVstars_user.groupby('name').count()
print(MTVstars_tweeters_name.sort(desc('count')).show())

In [ ]:
##check the number of verified accounts
MTVstars_ver = MTVstars_user.groupby('verified').count()
print(MTVStars_ver_only)

In [ ]:
##subset the users dataframe with tweet text and user name only
MTVStars_new=MTVStars_df.select("text","user.name")

##check for only verfied accounts
MTVStars_top1 = MTVStars_new.where(col("name").isin({"#MTVStars Coldplay","MTVCpVote","NICKI MINAJ","✞NICKI MINAJ✞","Katy Perry"}))
##save the results to a csv
MTVStars_top1.toPandas().to_csv('MTVStars_top1.csv')

In [ ]:
##import libraries to create a wordcloud
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt

In [ ]:
##read the csv file created in the previous step
wordcloud_df = pd.read_csv("MTVStars_top1.csv")

##convert the text in tweets to string
text_strings = wordcloud_df["text"].to_string()

##tokenize the words 
text_tokens = nltk.word_tokenize(text_strings)

In [ ]:
##import the stopwords list remove the stopwords from tweet texts
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')

##remove the stopwords
wordcloud_words = ' '.join([token for token in text_tokens if token not in stopword_list])

##convert the words into lowercase
wordcloud_words = wordcloud_words.lower()
x=' '.join([token for token in text_tokens if token not in stopword_list])

In [ ]:
##create the word cloud for most tweeted words
def create_word_cloud(string):
    cloud = WordCloud(width = 500, height = 500,background_color = "white", max_words = 500, stopwords = set(STOPWORDS))
    cloud.generate(string)
    cloud.to_file("wordCloud6.png")
create_word_cloud(str(wordcloud_words))